In [ ]:
# imports
import torch
import sys
from pathlib import Path

path = Path.cwd().parents[1]
if str(path) not in sys.path:
    sys.path.append(str(path))

from src.baselineGP import *
from src.kernels import *
from src.dgps import *

seed = 0
n = 500
ntest = 100 
d = 5
noise = 0.5 
niter = 1000
learn_rate = 0.1
error_samples = 10**2
gp_samples = 10**2
kernel = "gaussian"

# main
    
torch.manual_seed(seed)

""" Fixed configs """
Kernel = GaussianKernel
single_kernel = False
quantiles = torch.linspace(0,1,101)
reg = 1e-3
force_PD = True

if kernel == "gaussian":
    Kernel = GaussianKernel
else:
    Kernel = GammaExponentialKernel

""" Draw data """
Z, V, Y, doZ, YdoZ, EYdoZ = Abelation(n, ntest, d, noise, doZlower = 0, doZupper = 1, mc_samples_EYdoZ = 10**4, seed = seed) 

""" Initialise model """
model = baselineGP(Kernel_A = Kernel, 
               Kernel_V = Kernel, 
               dim_A = Z.size()[1], 
               dim_V = V.size()[1], 
               single_kernel = single_kernel)

""" Train model """
model.train(Y,Z,V,niter,learn_rate, force_PD = force_PD, reg = reg)

""" Get Posterior mean """
EYdoZ_samples, EVdoZ_samples = model.marginal_post_sample(Y,V,Z,doZ, 
                                                        reg = reg, 
                                                        error_samples = error_samples, 
                                                        gp_samples = gp_samples)

""" Compute out of sample metrics """
upper_quantiles = 1-(1-quantiles)/2
lower_quantiles = (1-quantiles)/2
u = (upper_quantiles*(gp_samples-1)).int()
l = (lower_quantiles*(gp_samples-1)).int()
EY_u = EYdoZ_samples.sort(1)[0][:,u]
EY_l = EYdoZ_samples.sort(1)[0][:,l]

posterior_fraction = ((EY_u>=EYdoZ)*(EY_l<=EYdoZ)).float()   
rmse = ((EYdoZ - EYdoZ_samples.mean(1).reshape(ntest,1))**2).mean()**0.5

/nfs/ghome/live/danceh/Causal-GP/causal-KL-GP/src/dgps.py:28: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1724888750493/work/aten/src/ATen/native/TensorShape.cpp:3697.)
  VdoZ = (f_x(doZ,coefs_v)).T[:,:,None] @ torch.ones(mc_samples_EYdoZ).view(1,mc_samples_EYdoZ) + noise_distribution.sample((mc_samples_EYdoZ,ntest)).T


iter 0 P(Y|V) loss:  tensor(79.0267)
iter 100 P(Y|V) loss:  tensor(-23.9146)
iter 200 P(Y|V) loss:  tensor(-24.0154)
iter 300 P(Y|V) loss:  tensor(-24.0235)
iter 400 P(Y|V) loss:  tensor(-24.0239)
iter 500 P(Y|V) loss:  tensor(-24.0240)
iter 600 P(Y|V) loss:  tensor(-24.0240)
iter 700 P(Y|V) loss:  tensor(-24.0240)
iter 800 P(Y|V) loss:  tensor(-24.0239)
iter 900 P(Y|V) loss:  tensor(-24.0240)
